# 1.- ETL
## 1.1 PASO 1. Proceso de limpieza de datos segun los requerimientos mínimos según Henry
1. Eliminar los campos que no se utilizaran: 'video','imdb_id','adult', 'original_title', 'poster_path', 'homepage'.
2. Dar formato de AAAA-mm-dd de haber fecha en el campo 'release_date' y crear el campo 'release_year'.
3. Eliminar los valores nulos del campo 'release_date'.
4. Eliminar los valores nulos de los campos 'revenue', 'budget' deben ser rellenados por el número 0.
5. Crear el campo 'return' con los campos 'revenue' y 'budget', dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.
6. Desanidar los campos: 'belongs_to_collection', 'genres', 'production_companies', 'production_countries' y 'spoken_languages'.

In [1]:
import pandas as pd
import numpy as np
import os

data_movies = pd.read_csv('D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Datasets\movies_dataset.csv')
data_movies.rename(columns={'id':'movie_id'}, inplace=True) #Cambiamos el id a movie_id para poder usarlo como primary key
data_movies.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   movie_id               45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

C:\Users\Losse\AppData\Local\Temp\ipykernel_27136\2093578661.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_movies = pd.read_csv('D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Datasets\movies_dataset.csv')


1. Eliminar los campos que no se utilizaran: 'video','imdb_id','adult', 'original_title', 'poster_path', 'homepage'.

In [2]:
columnas_excluidas_henry = ['video','imdb_id','adult','original_title','poster_path', 'homepage'] # Columnas que no serán utilizadas segun Henry
columnas_excluidas_criterio_personal = ['tagline', 'status']    #Columnas que no seran utilizadas segun criterio personal    
columnas_desanidar=['belongs_to_collection','genres','production_companies','production_countries','spoken_languages'] #Columnas que requieren ser desanidadas

data_movies_subset = data_movies.drop(list(columnas_excluidas_henry+columnas_excluidas_criterio_personal+columnas_desanidar), axis=1)
data_movies_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45466 non-null  object 
 1   movie_id           45466 non-null  object 
 2   original_language  45455 non-null  object 
 3   overview           44512 non-null  object 
 4   popularity         45461 non-null  object 
 5   release_date       45379 non-null  object 
 6   revenue            45460 non-null  float64
 7   runtime            45203 non-null  float64
 8   title              45460 non-null  object 
 9   vote_average       45460 non-null  float64
 10  vote_count         45460 non-null  float64
dtypes: float64(4), object(7)
memory usage: 3.8+ MB


In [3]:
#data_movies_subset['movie_id']=pd.to_numeric(data_movies_subset['movie_id'])

data_movies_subset.iloc[19730]['movie_id'] 

'1997-08-20'

In [4]:

# identificamos las filas que contienen datos tipo fecha en la columna de nuestro primary key: pelicula_id
def check_type(x):
     try: 
        return type(eval(x)) 
     except Exception as e:
        return type(x)

data_movies_subset.loc[data_movies_subset['movie_id'].map(check_type)!=int]

,budget,movie_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count
19730,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,1997-08-20,104.0,Released,NaN,1,NaN,NaN,NaN,NaN,NaN
29503,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,2012-09-29,68.0,Released,NaN,12,NaN,NaN,NaN,NaN,NaN
35587,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,2014-01-01,82.0,Released,Beware Of Frost Bites,22,NaN,NaN,NaN,NaN,NaN


In [5]:
# removemos las filas que contienen fecha en la columna de primary key
data_movies_subset.drop(
    data_movies_subset.loc[data_movies_subset['movie_id'].map(check_type)!=int].index,
    inplace=True)
print("Numero de filas que no contienen datos tipo fecha en la columna ['movie_id']:",
    data_movies_subset.loc[data_movies_subset['movie_id'].map(check_type)!=int].shape[0]
)

Numero de filas que no contienen datos tipo fecha en la columna ['movie_id']: 0


In [6]:
#data_movies_subset['movie_id']=pd.to_numeric(data_movies_subset['movie_id'])
#data_movies_subset.head()

2. Dar formato de AAAA-mm-dd de haber fecha en el campo 'release_date' y crear el campo 'release_year'.

In [7]:
#Convertimos el tipo de columna
columns_numeric=['movie_id','budget','revenue','popularity','vote_average','vote_count','runtime']
columns_date=['release_date']

data_movies_subset[columns_numeric]=data_movies_subset[columns_numeric].apply(pd.to_numeric) # convertimos a tipo numerico
data_movies_subset[columns_date]=data_movies_subset[columns_date].apply(pd.to_datetime) # convertimos a tipo fecha 

In [8]:
data_movies_subset_limpia=data_movies_subset.convert_dtypes() # volvemos a evaluar el df
data_movies_subset_limpia.info()
data_movies_subset_limpia.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45463 non-null  Int64         
 1   movie_id           45463 non-null  Int64         
 2   original_language  45452 non-null  string        
 3   overview           44509 non-null  string        
 4   popularity         45460 non-null  Float64       
 5   release_date       45376 non-null  datetime64[ns]
 6   revenue            45460 non-null  Int64         
 7   runtime            45203 non-null  Int64         
 8   title              45460 non-null  string        
 9   vote_average       45460 non-null  Float64       
 10  vote_count         45460 non-null  Int64         
dtypes: Float64(2), Int64(5), datetime64[ns](1), string(3)
memory usage: 4.5 MB


,budget,movie_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81,Toy Story,7.7,5415
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104,Jumanji,6.9,2413
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0,101,Grumpier Old Men,6.5,92


3. Eliminar los valores nulos del campo 'release_date'.

In [9]:
# instruccion 1: Los valores nulos del campo release date deben eliminarse
data_movies_subset_limpia['release_date'].isna().any()
print("Cantidad de valores nulos en ['release_date']:",
    data_movies_subset_limpia.loc[data_movies_subset_limpia['release_date'].isna()].shape[0])
data_movies_subset_limpia['release_date'].isna()
rows_release_date_na = data_movies_subset_limpia.loc[data_movies_subset_limpia['release_date'].isna()].index
data_movies_subset_limpia.drop(rows_release_date_na, inplace=True)
print("Cantidad de valores nulos en ['release_date'] despues del drop:",
    data_movies_subset_limpia.loc[data_movies_subset_limpia['release_date'].isna()].shape[0])
#Crear el 'release_year'
data_movies_subset_limpia['release_year'] = data_movies_subset_limpia['release_date'].dt.year

Cantidad de valores nulos en ['release_date']: 87
Cantidad de valores nulos en ['release_date'] despues del drop: 0


4. Eliminar los valores nulos de los campos 'revenue', 'budget' deben ser rellenados por el número 0.

In [10]:
# Reemplazar valores nulo de las columnas 'revenue' y 'budget' con "0"
data_movies_subset['revenue'] = data_movies_subset['revenue'].fillna(0)
data_movies_subset['budget'] = data_movies_subset['budget'].fillna(0)
# Cantidad de valores nulos en ['budget','revenue']
data_movies_subset[['budget','revenue']].isna().sum()

budget     0
revenue    0
dtype: int64

5. Crear el campo 'return' con los campos 'revenue' y 'budget', dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [11]:
#Creación de columna 'return' = 'revenue' / 'budget'
data_movies_subset_limpia['return'] = data_movies_subset_limpia['revenue']/data_movies_subset_limpia['budget']

#Reemplazo de valores nulos e indefinidos con '0'
data_movies_subset_limpia['return'] = data_movies_subset_limpia['return'].replace([np.inf, -np.inf], 0).fillna(0)
data_movies_subset_limpia.info()

print("Hay valores nulos en cualquiera de ['revenue','budget','return']:", any(data_movies_subset_limpia[['revenue','budget','return']].isna().any()))


<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45376 non-null  Int64         
 1   movie_id           45376 non-null  Int64         
 2   original_language  45365 non-null  string        
 3   overview           44435 non-null  string        
 4   popularity         45376 non-null  Float64       
 5   release_date       45376 non-null  datetime64[ns]
 6   revenue            45376 non-null  Int64         
 7   runtime            45130 non-null  Int64         
 8   title              45376 non-null  string        
 9   vote_average       45376 non-null  Float64       
 10  vote_count         45376 non-null  Int64         
 11  release_year       45376 non-null  int32         
 12  return             45376 non-null  Float64       
dtypes: Float64(3), Int64(5), datetime64[ns](1), int32(1), string(3)
me

In [12]:
# Removemos duplicados
data_movies_subset_limpia.drop_duplicates(inplace=True) # no funciona; deben existir filas que no son exactamente iguales.
print("Numero de peliculas unicas:",data_movies_subset_limpia['movie_id'].nunique(),"\n")
print("Numero de registros (filas):",data_movies_subset_limpia.shape[0],"\n")
print("Numero de peliculas con informacion inconsiste:",data_movies_subset_limpia.shape[0]-data_movies_subset_limpia['movie_id'].nunique(),"\n")

Numero de peliculas unicas: 45346 

Numero de registros (filas): 45359 

Numero de peliculas con informacion inconsiste: 13 



In [13]:
# para una misma pelicula existen valores distintos.
data_movies_subset_inconsistent=data_movies_subset_limpia.loc[data_movies_subset_limpia.duplicated(subset=['movie_id'], keep=False)]
print("Numero de filas con informacion duplicada en algunas columnas:",
    data_movies_subset_inconsistent.shape[0])
print("Numero de peliculas unicas con informacion duplicadas en algunas columnas:",
    data_movies_subset_inconsistent['movie_id'].nunique())
data_movies_subset_inconsistent.sort_values(['movie_id']).head(4)

Numero de filas con informacion duplicada en algunas columnas: 26
Numero de peliculas unicas con informacion duplicadas en algunas columnas: 13


,budget,movie_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return
5865,30000000,4912,en,"Television made him famous, but his biggest hi...",11.331072,2002-12-30,33013805,113,Confessions of a Dangerous Mind,6.6,281,2002,1.10046
33826,30000000,4912,en,"Television made him famous, but his biggest hi...",7.645827,2002-12-30,33013805,113,Confessions of a Dangerous Mind,6.6,281,2002,1.10046
44821,16000000,10991,ja,When Molly Hale's sadness of her father's disa...,6.480376,2000-07-08,68411275,93,Pokémon: Spell of the Unknown,6.0,144,2000,4.275705
4114,16000000,10991,ja,When Molly Hale's sadness of her father's disa...,10.264597,2000-07-08,68411275,93,Pokémon: Spell of the Unknown,6.0,143,2000,4.275705


In [14]:
# Solucionamos tomando la primera entrada para cada pelicula, y descartamos el resto.
data_movies_subset_limpia_sindups=data_movies_subset_limpia.loc[~data_movies_subset_limpia.duplicated(subset=['movie_id'], keep='first')]
print("Numero de peliculas unicas:",data_movies_subset_limpia_sindups['movie_id'].nunique(),"\n")
print("Numero de registros (filas):",data_movies_subset_limpia_sindups.shape[0],"\n")
print("Numero de peliculas con informacion inconsiste:",
      data_movies_subset_limpia_sindups.shape[0]-data_movies_subset_limpia_sindups['movie_id'].nunique(),"\n")

Numero de peliculas unicas: 45346 

Numero de registros (filas): 45346 

Numero de peliculas con informacion inconsiste: 0 



Guardamos el dataset limpio

In [15]:
# Nuestro dataset final
print("Universo de peliculas con datos unicos:",data_movies_subset_limpia_sindups['movie_id'].nunique(),"\n")
data_movies_subset_limpia_sindups.to_csv(os.path.join("D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Movies\Pipeline","data_movies_subset_limpia.csv"))
data_movies_subset_limpia_sindups.info()
data_movies_subset_limpia_sindups.head(2)

Universo de peliculas con datos unicos: 45346 

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45346 non-null  Int64         
 1   movie_id           45346 non-null  Int64         
 2   original_language  45335 non-null  string        
 3   overview           44405 non-null  string        
 4   popularity         45346 non-null  Float64       
 5   release_date       45346 non-null  datetime64[ns]
 6   revenue            45346 non-null  Int64         
 7   runtime            45100 non-null  Int64         
 8   title              45346 non-null  string        
 9   vote_average       45346 non-null  Float64       
 10  vote_count         45346 non-null  Int64         
 11  release_year       45346 non-null  int32         
 12  return             45346 non-null  Float64       
dtypes: Float64(3), Int

,budget,movie_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81,Toy Story,7.7,5415,1995,12.451801
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104,Jumanji,6.9,2413,1995,4.043035
